In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = pd.read_excel('./gpt/FAQ/한난_23하 공채_FAQ_최종_230816_수정.xlsx', engine='openpyxl')
df.dropna(axis=0, inplace=True)

# cols = df.iloc[0, 1:]
df = df[1:].reset_index(drop=True)
df.columns = ['순번', '구분', '질문', '답변']
df

,순번,구분,질문,답변
0,1,공통,신입직원의 신분은 어떻게 되나요?,"(일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, 전환평가를 거쳐 60점 이상이면 수습 해제 후 해당 직급 6급(을), 7급으로 전환됩니다. 수습기간 동안에는 인턴사원(8급수준)의 급여(약 179만원/월)가 지급됩니다.\n\n(전문직) 별정직(기간제, 1년) 근무 후 계약기간 종료 시점에 성과평가를 통해 정규직(일반직 4급(병) 수준)으로 전환 예정입니다."
1,2,공통,모집 직급별 초임연봉은 얼마인가요?,"신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다. (성과상여금 및 기타수당 제외, 세전기준)"
2,3,공통,입사 후 근무지역은 어디인가요?,본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영계획에 따라 배치됩니다. 희망지역에 배치되지 않을 수 있습니다.
3,4,공통,합숙소 제공이 되나요?,"일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 경우 합숙소 입주대상으로 보고 있으나, 세부 자격사항은 내규를 따릅니다."
4,5,공통,채용 과정에서 경력을 우대받을 수 있나요?,"전문직(회계사) 분야 외 채용에서는 경력과 관련된 자격 및 우대사항은 없으며, 경력관련 경험을 입사지원서에 기술할 수 있습니다. 다만, 입사 후 경력증빙서류 제출 시 공사규정에 따라 호봉산정에 반영은 가능합니다."
5,6,공통,채용 과정에서 석박사 학위를 우대받을 수 있나요?,신입직원 채용 과정에서는 석박사 학위 보유에 따른 우대사항은 없습니다.
6,7,공통,병역 미필자가 지원 가능한가요?,"고졸인재분야는 미필자도 지원 가능하나, 그 외 분야의 경우 병역필 또는 면제자만 지원가능 합니다. 여성 지원자는 미해당에 해당하며, 해당 사실에 대한 채용 시 차별 또는 불이익은 없습니다. 입사 후 군입대 시 복무기간은 휴직처리 되며, 휴직기간 동안의 급여는 없습니다. 다만, 복무완료 시 복무기간은 근속연수에 가산됩니다."
7,8,공통,전형과정에서 제출하는 정보 및 서류로 나이나 성별 등이 노출될 수 있지 않나요?,"전형과정에서 제출하는 정보 및 서류는 자격확인 또는 본인확인 등의 목적으로만 활용됩니다. 성명, 나이, 성별 등의 블라인드 대상 개인정보는 평가위원에게 공개되지 않습니다."
8,9,공통,현재 재학중인 학생도 지원할 수 있나요?,"고졸인재분야는 전문대 및 4년제 대학 졸업자 또는 졸업예정자 지원 불가하나, 그 외 분야의 경우 입사지원 시 학력의 제한이 없으므로 지원자격요건 충족 시 누구든지 지원이 가능합니다. 최종 합격자 결정 후 임용발령일부터 근무 가능해야 하며, 학교 졸업 후 입사 등 별도조치는 불가합니다."
9,10,공통,이전 전형 점수가 다음 전형에 반영되나요?,전형단계별 합격자 결정방식은 제로 베이스(Zero-base) 방식을 적용합니다. 자세한 전형단계별 배수는 채용공고의 전형절차를 확인하시기 바랍니다.


In [133]:
for i in df.iterrows():
    print("기준질문 :", i[1]['질문'])
    print("답변 :", i[1]['답변'])
    
    print("---"*30)

기준질문 : 신입직원의 신분은 어떻게 되나요?
답변 : (일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, 전환평가를 거쳐 60점 이상이면 수습 해제 후 해당 직급 6급(을), 7급으로 전환됩니다. 수습기간 동안에는 인턴사원(8급수준)의 급여(약 179만원/월)가 지급됩니다.

(전문직) 별정직(기간제, 1년) 근무 후 계약기간 종료 시점에 성과평가를 통해 정규직(일반직 4급(병) 수준)으로 전환 예정입니다.
------------------------------------------------------------------------------------------
기준질문 : 모집 직급별 초임연봉은 얼마인가요?
답변 : 신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다. (성과상여금 및 기타수당 제외, 세전기준)
------------------------------------------------------------------------------------------
기준질문 : 입사 후 근무지역은 어디인가요?
답변 : 본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영계획에 따라 배치됩니다. 희망지역에 배치되지 않을 수 있습니다.
------------------------------------------------------------------------------------------
기준질문 : 합숙소 제공이 되나요?
답변 : 일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 경우 합숙소 입주대상으로 보고 있으나, 세부 자격사항은 내규를 따릅니다.
------------------------------------------------------------------------------------------
기준질문 : 채용 과정에서 경력을 우대받을 수 있나요?
답변 : 전문직(회계사) 분야 외 채용에서는 경력과 

In [135]:
print(df.to_json(orient='records',force_ascii=False))

[{"순번":1,"구분":"공통","질문":"신입직원의 신분은 어떻게 되나요?","답변":"(일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, 전환평가를 거쳐 60점 이상이면 수습 해제 후 해당 직급 6급(을), 7급으로 전환됩니다. 수습기간 동안에는 인턴사원(8급수준)의 급여(약 179만원\/월)가 지급됩니다.\n\n(전문직) 별정직(기간제, 1년) 근무 후 계약기간 종료 시점에 성과평가를 통해 정규직(일반직 4급(병) 수준)으로 전환 예정입니다."},{"순번":2,"구분":"공통","질문":"모집 직급별 초임연봉은 얼마인가요?","답변":"신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다. (성과상여금 및 기타수당 제외, 세전기준)"},{"순번":3,"구분":"공통","질문":"입사 후 근무지역은 어디인가요?","답변":"본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영계획에 따라 배치됩니다. 희망지역에 배치되지 않을 수 있습니다."},{"순번":4,"구분":"공통","질문":"합숙소 제공이 되나요?","답변":"일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 경우 합숙소 입주대상으로 보고 있으나, 세부 자격사항은 내규를 따릅니다."},{"순번":5,"구분":"공통","질문":"채용 과정에서 경력을 우대받을 수 있나요?","답변":"전문직(회계사) 분야 외 채용에서는 경력과 관련된 자격 및 우대사항은 없으며, 경력관련 경험을 입사지원서에 기술할 수 있습니다. 다만, 입사 후 경력증빙서류 제출 시 공사규정에 따라 호봉산정에 반영은 가능합니다."},{"순번":6,"구분":"공통","질문":"채용 과정에서 석박사 학위를 우대받을 수 있나요?","답변":"신입직원 채용 과정에서는 석박사 학위 보유에 따른 우대사항은 없습니다."},{"순번":7,"구분":"공통","질문":"병역 미필자가 지원 가능한가요?","답변":"고졸인재분야는 미필

In [107]:
df['질문']

0                                    신입직원의 신분은 어떻게 되나요?
1                                   모집 직급별 초임연봉은 얼마인가요?
2                                     입사 후 근무지역은 어디인가요?
3                                          합숙소 제공이 되나요?
4                               채용 과정에서 경력을 우대받을 수 있나요?
5                           채용 과정에서 석박사 학위를 우대받을 수 있나요?
6                                     병역 미필자가 지원 가능한가요?
7          전형과정에서 제출하는 정보 및 서류로 나이나 성별 등이 노출될 수 있지 않나요?
8                                현재 재학중인 학생도 지원할 수 있나요?
9                               이전 전형 점수가 다음 전형에 반영되나요?
10                                      신체검사 기준은 무엇인가요?
11                                         신원조회는 무엇인가요?
12                개인사정으로 전형과정에서 놓친 부분이 있는데 감안해주실 수 있나요?
13                                        경쟁률이 어떻게 되나요?
14                                   과거 채용정보를 조회하고 싶어요.
15                                   모집분야별 중복지원이 가능한가요?
16                입사지원 오류로 입사지원서를 제출하지 못했는데 추가제출 가능한가요?
17                              입사지원서를 수정 또는 삭제할

In [113]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)
    
def extract_vector(response_subj, response_score, final_embedded):
    ##### 차원축소 안한거
    # datas_subj = [str(datas.embedding) for datas in response_subj.data]
    # datas_score = [str(datas.embedding) for datas in response_score.data]
    # embedding_series_subj = pd.Series(datas_subj)
    # embedding_series_score = pd.Series(datas_score)
    
    ##### 차원축소 한거
    datas_subj = [datas.embedding for datas in response_subj.data]
    datas_score = [datas.embedding for datas in response_score.data]
    embedding_series_subj = pd.Series(datas_subj).apply(lambda x : str(normalize_l2(x[:512]).tolist()))
    embedding_series_score = pd.Series(datas_score).apply(lambda x : str(normalize_l2(x[:512]).tolist()))
    
    concat_embedded = pd.concat([embedding_series_subj, embedding_series_score], axis = 1).reset_index(drop=True)
    final_embedded = pd.concat([final_embedded, concat_embedded]).reset_index(drop=True)
    return final_embedded


load_dotenv("/mnt/c/Users/USER/Desktop/nam/gpt/.env")
api_key = os.getenv('key')
client = OpenAI(api_key=api_key)
model = "text-embedding-3-small"


response = client.embeddings.create(
    input=list(df['질문']),
    model=model
)
    
    
#     final_embedded = extract_vector(response_subj, response_score, final_embedded)

In [118]:
df['embedded_vector'] = [i.embedding for i in response.data]
df

,순번,구분,질문,답변,embedded_vector
0,1,공통,신입직원의 신분은 어떻게 되나요?,"(일반·사회형평 및 고졸인재 분야) 8급 수습직원으로 약 3개월간의 수습기간 후, ...","[0.025579623878002167, 0.0722941905260086, 0.0..."
1,2,공통,모집 직급별 초임연봉은 얼마인가요?,"신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다...","[-0.0178645309060812, 0.048357851803302765, 0...."
2,3,공통,입사 후 근무지역은 어디인가요?,본사를 비롯한 전국 사업소 및 지사에서 근무하게 됩니다. 근무지는 우리공사 인력운영...,"[-0.03703274950385094, 0.03369060531258583, 0...."
3,4,공통,합숙소 제공이 되나요?,"일반적으로 주민등록상 주소가 근무지역 외이며, 근무지역 내 무주택(배우자 포함)인 ...","[-0.02958998829126358, 0.017489179968833923, 0..."
4,5,공통,채용 과정에서 경력을 우대받을 수 있나요?,"전문직(회계사) 분야 외 채용에서는 경력과 관련된 자격 및 우대사항은 없으며, 경력...","[-0.019404234364628792, -0.03193490579724312, ..."
5,6,공통,채용 과정에서 석박사 학위를 우대받을 수 있나요?,신입직원 채용 과정에서는 석박사 학위 보유에 따른 우대사항은 없습니다.,"[-0.025783948600292206, -0.006684326101094484,..."
6,7,공통,병역 미필자가 지원 가능한가요?,"고졸인재분야는 미필자도 지원 가능하나, 그 외 분야의 경우 병역필 또는 면제자만 지...","[-0.03362102061510086, -0.014997242018580437, ..."
7,8,공통,전형과정에서 제출하는 정보 및 서류로 나이나 성별 등이 노출될 수 있지 않나요?,전형과정에서 제출하는 정보 및 서류는 자격확인 또는 본인확인 등의 목적으로만 활용됩...,"[0.02264249511063099, 0.00933252926915884, 0.0..."
8,9,공통,현재 재학중인 학생도 지원할 수 있나요?,"고졸인재분야는 전문대 및 4년제 대학 졸업자 또는 졸업예정자 지원 불가하나, 그 외...","[-0.04835692420601845, 0.020624758675694466, -..."
9,10,공통,이전 전형 점수가 다음 전형에 반영되나요?,전형단계별 합격자 결정방식은 제로 베이스(Zero-base) 방식을 적용합니다. 자...,"[-0.045118097215890884, 0.01041724719107151, 0..."


In [121]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model=model): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

model = "text-embedding-3-small"
def search_docs(df, user_query):
    embedding = get_embedding(
        user_query,
        model=model # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    # small_emb = normalize_l2(embedding[:512])
    # if col == 'subj':
    df["similarities"] = df['embedded_vector'].apply(lambda x: cosine_similarity(x, embedding))
    res = (
        df.sort_values("similarities", ascending=False)
        .head(5)
    )
    # else:
    #     df["similarities"] = df.score_list.apply(lambda x: cosine_similarity(x, small_emb))
    return res
    # if to_print:
    #     display(res)
    # return res


res = search_docs(df, '몇 급으로 들어가나요?')
res

,순번,구분,질문,답변,embedded_vector,similarities
19,20,입사지원,자격증은 몇 개까지 제출 가능한가요?,"일반, 사회형평 및 고졸분야 전의 경우 공통자격 최대 3개, 직무자격 최대 2개만 ...","[0.012132611125707626, 0.03853888437151909, 0....",0.300886
1,2,공통,모집 직급별 초임연봉은 얼마인가요?,"신입직원 초임연봉은 6급(을)은 3,450만원 수준, 7급 2,650만원 수준입니다...","[-0.0178645309060812, 0.048357851803302765, 0....",0.266425
28,29,필기전형,필기전형 응시에 따른 응시지원비가 있나요?,필기전형 응시에 따른 응시지원비는 별도로 제공되지 않습니다.,"[0.005977964028716087, -0.012954361736774445, ...",0.224291
4,5,공통,채용 과정에서 경력을 우대받을 수 있나요?,"전문직(회계사) 분야 외 채용에서는 경력과 관련된 자격 및 우대사항은 없으며, 경력...","[-0.019404234364628792, -0.03193490579724312, ...",0.220448
29,30,면접전형,면접전형 응시에 따른 응시지원비가 있나요?,면접전형 응시에 따른 교통비(약 5만원)를 별도로 제공할 예정입니다.\n응시자의 주...,"[-0.019649365916848183, 0.008223996497690678, ...",0.218082
